## Ipython and the "pythranize_at_import" mode

Here, we show how easy it is to work with modules using Pythran from IPython. We also show a little bit of how it works internally.

In [1]:
import time
from pathlib import Path

from fluidpythran import set_pythranize_at_import

# an internal object to get more information on what is going on under the hood 
from fluidpythran.aheadoftime import modules

We define a small function to display the state of a FluidPythran object representing a module:

In [2]:
def get_fp_mod_display_state(name):
    fp = modules[name]
    print(
        "(is_transpiled, is_compiling, is_compiled) =", 
        (fp.is_transpiled, fp.is_compiling, fp.is_compiled),
    )
    return fp

We first write in a small Python module which defines a function using FluidPythran. We are going to run this module and to modify it during the Ipython session.

In [3]:
src = """
from fluidpythran import boost

@boost
def func(n: int):
    return 2 * n

if __name__ == "__main__":
    print("result func(1):", func(1))
"""

with open("tmp.py", "w") as file:
    file.write(src)

Let's first clean all FluidPythran cache files...

In [4]:
paths = Path("__pythran__").glob("tmp*.*")
for path in paths:
    path.unlink()

We first use FluidPythran in the default mode for which Pythran is not used at run time.

In [5]:
set_pythranize_at_import(False)

In [6]:
run tmp.py

result func(1): 2


In [7]:
run tmp.py

result func(1): 2


Let's now switch on the FluidPythran mode with Pythran compilations at import time.

In [8]:
set_pythranize_at_import(True)

We then rerun the tmp.py module:

In [9]:
%run tmp.py

Running fluidpythran on file /home/users/augier3pi/Dev/fluidpythran/doc/ipynb/executed/tmp.py... Done!
Launching Pythran to compile a new extension...
result func(1): 2


In [10]:
ls --color=never __pythran__

__pycache__/  tmp.py


Note that we used `ls --color=never` just because colors in the terminal are not well rendered in html produced by Jupyter nbconvert...  

In [11]:
fp_mod = get_fp_mod_display_state("tmp")

(is_transpiled, is_compiling, is_compiled) = (True, True, False)


We see that FluidPythran created a Pythran file at import and launch the compilation. We can still rerun the module and it works as without FluidPythran:

In [12]:
%run tmp.py

result func(1): 2


While it's compiling, let's wait and call the function.

In [13]:
while fp_mod.is_compiling:
    print(func(1), end=",")
    time.sleep(1)

2,2,2,2,2,

Ok so here, we see that Pythran compilations are quite long! In particular compared to Numba, PyPy or Julia JIT compilations! But after these few seconds, it should very efficient...

In [14]:
fp_mod = get_fp_mod_display_state("tmp")

(is_transpiled, is_compiling, is_compiled) = (True, False, True)


Now the compilation is done. Let's check that there is a new extension file:

In [15]:
ls --color=never __pythran__

__pycache__/                                                          tmp.py
tmp_26ab178b59974a4f661ae5399989c3f1.cpython-36m-x86_64-linux-gnu.so


In [16]:
print("use the compiled functions (now it should be faster):", func(1))

use the compiled functions (now it should be faster): 2


In [17]:
%run tmp.py

result func(1): 2


In [18]:
print("use the compiled functions again:", func(1))

use the compiled functions again: 2


Let's update the code of the module and see how FluidPythran is going to update the Pythran file and the extension!

In [19]:
src1 = """
from fluidpythran import boost

@boost
def func(n: int):
    return 4 * n

if __name__ == "__main__":
    print("result func(1):", func(1))
"""

with open("tmp.py", "w") as file:
    file.write(src1)

In [20]:
%run tmp.py

Running fluidpythran on file /home/users/augier3pi/Dev/fluidpythran/doc/ipynb/executed/tmp.py... Done!
Launching Pythran to compile a new extension...
result func(1): 4


In [21]:
%run tmp.py

result func(1): 4


In [22]:
fp_mod = get_fp_mod_display_state("tmp")

(is_transpiled, is_compiling, is_compiled) = (True, True, False)


In [23]:
while fp_mod.is_compiling:
    print(func(1), end=",")
    time.sleep(1)

4,4,4,4,4,

In [24]:
ls --color=never __pythran__

__pycache__/
tmp_26ab178b59974a4f661ae5399989c3f1.cpython-36m-x86_64-linux-gnu.so
tmp_5a9e4142c349b2479b238a14d1369378.cpython-36m-x86_64-linux-gnu.so
tmp.py


In [25]:
fp_mod = get_fp_mod_display_state("tmp")

print("use the compiled functions:", func(1))

(is_transpiled, is_compiling, is_compiled) = (True, False, True)
use the compiled functions: 4


In [26]:
%run tmp.py

result func(1): 4


We now update the module such that the Pythran functions are the same as in the first example.

In [27]:
src2 = """

from fluidpythran import boost


@boost
def func(n: int):
    return 4 * n
    

def func_hello():
    print("hello")


if __name__ == "__main__":
    print("result func(1):", func(1))
"""


with open("tmp.py", "w") as file:
    file.write(src2)

In [28]:
%run tmp.py

Running fluidpythran on file /home/users/augier3pi/Dev/fluidpythran/doc/ipynb/executed/tmp.py... Done!
result func(1): 4


In [29]:
fp_mod = get_fp_mod_display_state("tmp")

(is_transpiled, is_compiling, is_compiled) = (True, False, True)


Great! No need for compilation because the extension has been cached!

In [30]:
while fp_mod.is_compiling:
    print(func(1), end=",")
    time.sleep(1)

In [31]:
ls --color=never __pythran__

__pycache__/
tmp_26ab178b59974a4f661ae5399989c3f1.cpython-36m-x86_64-linux-gnu.so
tmp_5a9e4142c349b2479b238a14d1369378.cpython-36m-x86_64-linux-gnu.so
tmp.py


In [32]:
fp_mod = get_fp_mod_display_state("tmp")

print("use the compiled functions:", func(1))

(is_transpiled, is_compiling, is_compiled) = (True, False, True)
use the compiled functions: 4


#### Summary

With the "pythranize_at_import" mode (set with the function `set_pythranize_at_import` or by the environment variable `FLUIDPYTHRANIZE_AT_IMPORT`), we can just work interactively modifying Python files and running them in IPython as without FluidPythran. FluidPythran automatically takes care of creating Pythran files and calling Pythran to create the extensions.